### Scrabble Game
Write a Python program that takes a Scrabble rack as a function argument and prints all "valid Scrabble English" words that can be constructed from that rack, along with their Scrabble scores, sorted by score. "valid Scrabble English" words are provided in the data source below. A Scrabble rack is made up of 2 to 7 characters.

- Allow anywhere from 2-7 character tiles (letters A-Z, upper or lower case) to be inputted. 
- Do not restrict the number of same tiles (e.g., a user is allowed to input ZZZZZQQ).
  - You need to handle input errors from the user and suggest what that error might be caused by and how to fix it (i.e., a helpful error message). **Return** this error message as a string from the run_scrabble function (do not raise an exception).
- Implement wildcards as either `*` or `?`. There can be a total of **only** two wild cards in any user input (that is, one of each character: one `*` and one `?`). Only use the `*` and `?` as wildcard characters. A wildcard character can take any value A-Z. Replace the wildcard symbol with the letter in your answer (see the second example below). 
  - Wildcard characters are scored as 0 points, just like in the real Scrabble game. A word that just consists of two wildcards can be made, should be outputted and scored as 0 points. 
  - In a wildcard case where the same word can be made with or without the wildcard, display the highest score. For example: given the input 'I?F', the word 'if' can be made with the wildcard '?F' as well as the letters 'IF'. Since using the letters 'IF' scores higher, display that score.
- For partial credit, your program should take less than one minute to run with 2 wildcards in the input. For full credit, the program needs to run with 2 wildcards in less than 30 seconds.

#### The Data
The file: http://courses.cms.caltech.edu/cs11/material/advjava/lab1/sowpods.zip contains all "valid Scrabble English" words in the official words list, one word per line. You should download the word file and keep it in your repository so that the program is standalone (instead of accessing it over the web from Python).

This `score_word` function will take each word and return the score (scoring dictionary is described below).

In [1]:
def score_word(word, letters_not_scored):
    
    """
    Calculate the Scrabble score for a given word, considering wildcard characters.

    Args:
    - word (str): The word for which the score needs to be calculated.
    - letters_not_scored (dict): A dictionary containing letters replaced by wildcard characters in each word.

    Returns:
    - int: The Scrabble score for the word.
    """
   
      # Dictionary to map each letter to its Scrabble score
    scoring_dict = {"A": 1, "C": 3, "B": 3, "E": 1, "D": 2, "G": 2,
         "F": 4, "I": 1, "H": 4, "K": 5, "J": 8, "M": 3,
         "L": 1, "O": 1, "N": 1, "Q": 10, "P": 3, "S": 1,
         "R": 1, "U": 1, "T": 1, "W": 4, "V": 4, "Y": 4,
         "X": 8, "Z": 10}
        # Initialize the score to 0
    score = 0
        # Loop through each letter and or character in the word
    for letter in word:
    # If the letter is not a wildcard character, add its score
        if letter not in ('*', '?'):
            score += scoring_dict[letter.upper()]
    # Ensure the score is not negative
    score = max(score, 0)
    # Subtracting the score of letters replaced by wildcard characters
    for letter in letters_not_scored.get(word, ''):
        #print("letter is:",letter)
        if letter in scoring_dict:
            score -= scoring_dict[letter]
    # Ensure the score is not negative
    score = max(score, 0)
    return max(score, 0)

The `run_scrabble` function returns two items:
- 1) The **total** list of valid Scrabble words that can be constructed from the rack as (score, word) tuples, sorted by the score and then by the word alphabetically as shown in the first example below. All outputted words need to be in upper case.
- 2) The Total number of valid words as an integer

In [ ]:
from wordscore import score_word

def load_valid_words(file_path):
    # Load the valid words from the specified file and store them in a set
    with open(file_path, 'r') as file:
        valid_words = {line.strip().upper() for line in file}
    return valid_words

def run_scrabble(rack):
    """
    This function takes a Scrabble rack as input and returns all valid Scrabble English words that can be constructed
    from that rack, along with their Scrabble scores after subtracting the scores
     of the wild characters (*?) if included in rack, sorted by score.

    Args:
    - rack (str): The Scrabble rack, consisting of 2 to 7 characters (A-Z, upper or lower case),
                  and optionally including wildcard characters '*' or '?'.

    Returns:
    - tuple: A tuple containing a list of tuples (score, word) representing valid Scrabble words 
             constructed from the rack, sorted by score and then alphabetically, and an integer representing the 
             total number of valid words.
    """
    # Converting the rack to uppercase to ensure consistency
    rack = rack.upper()

    # Error validation
    if not 2 <= len(rack) <= 7:
        return "Error: The Scrabble Rack must have 2 to 7 characters."
    if rack.count('*') > 1 or rack.count('?') > 1:
        return "Error: The Scrabble Rack can have at most one '*' and one '?'."
    if not all(char.isalpha() or char in ['*', '?'] for char in rack):
        return "Error: The Scrabble Rack can only contain letters of the alphabet, '*' and '?'."

    # Load the valid words from the dictionary file
    valid_words = load_valid_words("sowpods.txt")

    # List to store valid words formed from the rack
    valid_word_list = []
    # Dictionary to store letters not scored for each valid word
    letters_not_scored = {}

    # Iterating over each word in the dictionary of valid words
    for word in valid_words:
        # Create a copy of the rack to manipulate
        temp = list(rack)
        count = 0
        letters_not_scored[word] = ''
        for char in word:
            if char in temp:
                count += 1
                temp.remove(char)
            elif '*' in temp:
                count += 1
                temp.remove('*')
                letters_not_scored[word] += char
            elif '?' in temp:
                count += 1
                temp.remove('?')
                letters_not_scored[word] += char
            else:
                break
        else:
            if len(word) == count:
                # If the loop completes (meaning all characters of the word were found in the rack),
                # and the length of the word is equal to the number of characters found in the rack,
                # add the word to the list of valid words then display in all caps
                valid_word_list.append(word.upper())

    # Calculate the score for each valid word plus subtracting the wild characters then storings them in a list of tuples
    valid_words_scored = [(score_word(word, letters_not_scored),  word) for word in valid_word_list]
    # Sort the list of valid word tuples by score (descending) and then by word (ascending)
    valid_words_scored.sort(key=lambda x: (-x[0], x[1]))
    
    # Return the valid words scored along with the total number of valid words
    return valid_words_scored, len(valid_words_scored)
